In [1]:
import json
import time
import numpy as np

In [3]:
context_corpus = json.load(open('dataset/paragraph_context.json', 'r', encoding='utf8')) 
question_corpus = json.load(open('dataset/question_context.json', 'r', encoding='utf8'))
train_labels_json = json.load(open('dataset/train_labels.json', 'r', encoding='utf8'))
test_labels_json = json.load(open('dataset/test_labels.json', 'r', encoding='utf8'))

In [4]:
from torch import nn
import torch
from transformers import ElectraTokenizer, ElectraModel, ElectraConfig, AdamW, get_linear_schedule_with_warmup
from torch.utils.data import DataLoader, Dataset

In [5]:
# monologg/kobert, "monologg/koelectra-base-v3-discriminator"

In [6]:
contexts = list(context_corpus.values())

In [7]:
class CreateDataset(Dataset):
    def __init__(self, context_corpus, question_corpus, labels):
        self.context_corpus = context_corpus
        self.question_corpus = question_corpus
        self.labels = list(labels.items())
        print(self.labels[0])
    
    def __len__(self):
        return len(self.labels)
        
    def __getitem__(self, idx):
        que_id, doc_id = self.labels[idx]
        return {'context': self.context_corpus[doc_id], 'question': self.question_corpus[que_id]}

In [8]:
class Model(nn.Module):
    def __init__(self, args):
        super().__init__()
        config = ElectraConfig.from_pretrained(args.model_name, local_file_only=True)
        self.model  = ElectraModel.from_pretrained(args.model_name, config=config)
        self.tokenizer = ElectraTokenizer.from_pretrained(args.model_name, local_file_only=True)
        
        self.punctation_idx = self.tokenizer.get_vocab()['.']
        self.pad_token_idx = self.tokenizer.pad_token_id
        self.mask_token_idx = self.tokenizer.mask_token_id
        self.d = self.tokenizer.get_vocab()['[unused0]']
        self.q = self.tokenizer.get_vocab()['[unused1]']
        self.linear = nn.Linear(config.hidden_size, 256)
        
        self.doc_maxlen = args.doc_maxlen
        self.query_maxlen = args.query_maxlen
        self.device = args.device
        self.criterion = nn.CrossEntropyLoss()
        
        
    def forward(self, feature):
        q_output = self.query(feature['question'])
        d_output = self.doc(feature['context'])
        prediction = self.similarity(q_output, d_output)
        print(prediction)
        loss = self.calc_loss(prediction)
        return loss
    
    def calc_loss(self, prediction):
        batch_size = prediction.shape[0]
        label = torch.arange(batch_size).to(self.device)
        return self.criterion(prediction, label)
        
    
    def similarity(self, q_output, d_output):
        # q_output = [batch, 128, 256]
        # d_output = [batch, seq_len, 256]
        prediction = torch.einsum('ijk,abk->iajb', q_output, d_output)
        prediction, _ = torch.max(prediction, dim=-1)
        prediction = torch.sum(prediction, dim=-1)
        return prediction
    
    
    def doc(self, D):
        inputs = self.tokenizer(D, return_tensors='pt', padding=True, truncation=True, max_length=self.doc_maxlen)
        
        input_ids = inputs['input_ids']
        attention_mask = inputs['attention_mask']
        
        batch = input_ids.shape[0]
        
        b = torch.LongTensor([self.d]* batch).view(-1, 1)
        a = torch.ones(size=(batch, 1))
        
        input_ids = torch.cat([input_ids[:, :1], b, input_ids[:, 1:]], dim=1).to(self.device)
        attention_mask = torch.cat([attention_mask[:, :1], a, attention_mask[:, 1:]], dim=1).to(self.device)
        
        punctation = input_ids
        
        model_input = {'input_ids': input_ids,
                      'attention_mask': attention_mask}
        
        output = self.model(**model_input)['last_hidden_state']
        output = self.linear(output)
        
        new_mask = attention_mask * (punctation != self.punctation_idx)
        output = output * new_mask.unsqueeze(-1)
        output = torch.nn.functional.normalize(output, p=2, dim=2)
        return output
    
    
    def query(self, Q):
        inputs = self.tokenizer(Q, return_tensors='pt', truncation=True, max_length=self.query_maxlen,
                               pad_to_max_length=True)
        
        input_ids = inputs['input_ids']
        attention_mask = inputs['attention_mask']
        
        input_ids = input_ids.masked_fill(attention_mask==self.pad_token_idx, self.mask_token_idx)
        
        batch = input_ids.shape[0]
        
        b = torch.LongTensor([self.d]* batch).view(-1, 1)
        a = torch.zeros(size=(batch, 1))
        
        input_ids = torch.cat([input_ids[:, :1], b, input_ids[:, 1:]], dim=1).to(self.device)
        new_mask = torch.ones_like(input_ids).to(self.device)
        
        punctation = input_ids
        
        model_input = {'input_ids': input_ids,
                      'attention_mask': new_mask}
        
        output = self.model(**model_input)['last_hidden_state']
        output = self.linear(output)
        output = torch.nn.functional.normalize(output, p=2, dim=2)
        return output


In [9]:
import easydict
from tqdm import tqdm
args = easydict.EasyDict({
    'model_name': 'monologg/koelectra-base-v3-discriminator',
    'doc_maxlen': 512-1,
    'query_maxlen': 128-1,
    'device': 'cuda',
    'epochs': 5,
    'warmup': 0.1,
    'batch_size': 16
})

In [10]:
model = torch.load('best.pt').to(args.device).eval()

In [13]:
class Document(Dataset):
    def __init__(self, corpus, test_labels_json):
        self.corpus = corpus
        self.label = self.create_label(corpus, test_labels_json)
        
    def create_label(self, corpus, test_labels_json):
        label = []
        for key in test_labels_json.keys():
            label.append(key)
        return label
            
    
    def __len__(self):
        return len(self.label)
    
    def __getitem__(self, idx):
        return {'context': self.corpus[self.label[idx]], 'context_id': self.label[idx]}

In [14]:
document_dataset = Document(context_corpus, test_labels_json)
document_dataloader = DataLoader(document_dataset, batch_size=1, shuffle=False)

In [15]:
len(document_dataloader)

24711

In [16]:
for i in document_dataloader:
    print(i)
    break

{'context': ['존경하는 국민 여러분, 농어민 여러분! ‘농정틀 전환을 위한 2019 타운홀미팅 보고대회’를 전통과 한식의 도시 전주에서 열게 되어 매우 뜻깊게 생각합니다. 특별히 농어업 인재양성의 산실 한국농수산대학의 젊은 후계자들과 함께해 더욱 기쁩니다. 우리 농어업의 새로운 미래가 청년들에게 희망이 되기를 기대합니다. 우리는 모두 농어민의 자손입니다. 우리는 선사 시대부터 벼농사와 어업을 함께하는 농경사회를 이루었고, 농어업을 통해 자연의 섭리를 배웠습니다. 우리가 어른을 공경하고 공동체를 중요하게 여기는 우수한 민족이 된 것도 농어업으로 형성된 협동정신이 있었기 때문입니다. 우리 민족의 정신과 뿌리도 농어촌에 있습니다. 전라북도에서 시작한 동학농민혁명은 농민 스스로 일어나 나라를 개혁하고자 했고, 그 정신이 의병활동과 3·1독립운동으로 이어져 대한민국임시정부 수립과 민주공화국 수립의 근간이 되었습니다. 많은 애국지사가 나라의 주인이 농민임을 천명하며 농촌 계몽운동으로 근대문명과 독립의 힘을 키웠습니다. 대한민국 발전의 근간도 농어촌이었습니다. 오늘 우리가 이룩한 눈부신 산업 발전도 농어촌의 뒷받침이 있었기에 가능했습니다. 하지만 그 과정에서 농어촌은 피폐해지고 도시와 격차가 커져온 것이 사실입니다. '], 'context_id': ['PARS_bEpJzknCai']}


In [17]:
document_vector = {}

In [18]:
for cnt, x in tqdm(enumerate(document_dataloader)):
    doc_vec = model.doc(x['context']).cpu().detach().numpy()[0]
    document_vector[x['context_id'][0]] = doc_vec

24711it [07:15, 56.81it/s]


In [19]:
list(document_vector.keys())[0]

'PARS_bEpJzknCai'

In [26]:
class Question(Dataset):
    def __init__(self, corpus, test_labels_json):
        self.corpus = corpus
        self.label = self.create_label(corpus, test_labels_json)
        
    def create_label(self, corpus, test_labels_json):
        label = []
        for key in test_labels_json.keys():
            for ques in test_labels_json[key]:
                label.append(ques)
        return label
    
    def __len__(self):
        return len(self.label)
    
    def __getitem__(self, idx):
        return {'question': self.corpus[self.label[idx]], 'question_id': self.label[idx]}

In [27]:
question_dataset = Question(question_corpus, test_labels_json)
question_dataloader = DataLoader(question_dataset, batch_size=1, shuffle=False)

In [28]:
for i in document_dataloader:
    print(i)
    break

{'context': ['존경하는 국민 여러분, 농어민 여러분! ‘농정틀 전환을 위한 2019 타운홀미팅 보고대회’를 전통과 한식의 도시 전주에서 열게 되어 매우 뜻깊게 생각합니다. 특별히 농어업 인재양성의 산실 한국농수산대학의 젊은 후계자들과 함께해 더욱 기쁩니다. 우리 농어업의 새로운 미래가 청년들에게 희망이 되기를 기대합니다. 우리는 모두 농어민의 자손입니다. 우리는 선사 시대부터 벼농사와 어업을 함께하는 농경사회를 이루었고, 농어업을 통해 자연의 섭리를 배웠습니다. 우리가 어른을 공경하고 공동체를 중요하게 여기는 우수한 민족이 된 것도 농어업으로 형성된 협동정신이 있었기 때문입니다. 우리 민족의 정신과 뿌리도 농어촌에 있습니다. 전라북도에서 시작한 동학농민혁명은 농민 스스로 일어나 나라를 개혁하고자 했고, 그 정신이 의병활동과 3·1독립운동으로 이어져 대한민국임시정부 수립과 민주공화국 수립의 근간이 되었습니다. 많은 애국지사가 나라의 주인이 농민임을 천명하며 농촌 계몽운동으로 근대문명과 독립의 힘을 키웠습니다. 대한민국 발전의 근간도 농어촌이었습니다. 오늘 우리가 이룩한 눈부신 산업 발전도 농어촌의 뒷받침이 있었기에 가능했습니다. 하지만 그 과정에서 농어촌은 피폐해지고 도시와 격차가 커져온 것이 사실입니다. '], 'context_id': ['PARS_bEpJzknCai']}


In [29]:
question_vector = {}

In [30]:
for cnt, x in tqdm(enumerate(question_dataloader)):
    ques_vec = model.query(x['question']).cpu().detach().numpy()[0]
    question_vector[x['question_id'][0]] = ques_vec

0it [00:00, ?it/s]C:\Users\Inha\anaconda3\envs\inha\lib\site-packages\transformers\tokenization_utils_base.py:2212: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(
46691it [12:01, 64.69it/s]


In [31]:
import heapq

In [32]:
question_ids_save = list(question_vector.keys())

batch_system = []
batch_id = []
batch = 128
for index in tqdm(range(0, len(question_ids_save), batch)):
    length = batch if index+batch <= len(question_ids_save) else len(question_ids_save) - index
    print(index, batch, length, len(question_ids_save))
    q_vector = np.zeros(shape=(length, 128, 256))
    dummy = []
    for i in range(index, index+length):
        q_vector[i-index] = question_vector[question_ids_save[i]]
        dummy.append(question_ids_save[i])
    batch_system.append(q_vector)
    batch_id.append(dummy)

  3%|██▏                                                                              | 10/365 [00:00<00:04, 87.71it/s]

0 128 128 46691
128 128 128 46691
256 128 128 46691
384 128 128 46691
512 128 128 46691
640 128 128 46691
768 128 128 46691
896 128 128 46691
1024 128 128 46691
1152 128 128 46691
1280 128 128 46691
1408 128 128 46691
1536 128 128 46691
1664 128 128 46691
1792 128 128 46691
1920 128 128 46691
2048 128 128 46691
2176 128 128 46691


  8%|██████▋                                                                          | 30/365 [00:00<00:03, 90.52it/s]

2304 128 128 46691
2432 128 128 46691
2560 128 128 46691
2688 128 128 46691
2816 128 128 46691
2944 128 128 46691
3072 128 128 46691
3200 128 128 46691
3328 128 128 46691
3456 128 128 46691
3584 128 128 46691
3712 128 128 46691
3840 128 128 46691
3968 128 128 46691
4096 128 128 46691
4224 128 128 46691
4352 128 128 46691
4480 128 128 46691
4608 128 128 46691


 14%|███████████                                                                      | 50/365 [00:00<00:03, 91.92it/s]

...

41472 128 128 46691
41600 128 128 46691
41728 128 128 46691
41856 128 128 46691
41984 128 128 46691
42112 128 128 46691
42240 128 128 46691
42368 128 128 46691
42496 128 128 46691
42624 128 128 46691
42752 128 128 46691
42880 128 128 46691
43008 128 128 46691
43136 128 128 46691
43264 128 128 46691
43392 128 128 46691
43520 128 128 46691
43648 128 128 46691


 97%|█████████████████████████████████████████████████████████████████████████████▌  | 354/365 [00:03<00:00, 90.29it/s]

43776 128 128 46691
43904 128 128 46691
44032 128 128 46691
44160 128 128 46691
44288 128 128 46691
44416 128 128 46691
44544 128 128 46691
44672 128 128 46691
44800 128 128 46691
44928 128 128 46691
45056 128 128 46691
45184 128 128 46691
45312 128 128 46691
45440 128 128 46691
45568 128 128 46691
45696 128 128 46691
45824 128 128 46691


100%|████████████████████████████████████████████████████████████████████████████████| 365/365 [00:04<00:00, 90.37it/s]

45952 128 128 46691
46080 128 128 46691
46208 128 128 46691
46336 128 128 46691
46464 128 128 46691
46592 128 99 46691


In [33]:
colbert_answer = {x: [] for x in question_ids_save}

for k in tqdm(document_vector.keys()):
    d_vector = torch.FloatTensor(document_vector[k]).unsqueeze(0)
    d_id = k

    for q_vector, b_id in zip(batch_system, batch_id):
        q_vector = torch.FloatTensor(q_vector)
        sim = model.similarity(q_vector.to('cuda'), d_vector.to('cuda'))
        
        for ans, s in zip(b_id, sim):
            heapq.heappush(colbert_answer[ans], (s.item(), d_id))
        
            if len(colbert_answer[ans]) > 150:
                heapq.heappop(colbert_answer[ans])

100%|█████████████████████████████████████████████████████████████████████████| 24711/24711 [40:52:58<00:00,  5.96s/it]


In [ ]:
len(question_ids_save)

In [40]:
test_labels = {}
for t_l in test_labels_json:
    for ques in test_labels_json[t_l]:
        test_labels[ques] = t_l

In [46]:
sorted(colbert_answer[list(colbert_answer.keys())[0]], key=lambda x:x[0])

[(99.36807250976562, 'PARS_22HkDLsm8e'),
 (99.38423156738281, 'PARS_OZ1Rl1RA5X'),
 (99.45043182373047, 'PARS_qe0PwLnjls'),
 (99.45416259765625, 'PARS_l62DRK0d7F'),
 (99.4552230834961, 'PARS_8Pf5bF1cWh'),
 (99.46266174316406, 'PARS_Z5vZd89MLk'),
 (99.46607971191406, 'PARS_XyF2vkLgRs'),
 (99.47541809082031, 'PARS_zSLroHKBUa'),
 (99.48851013183594, 'PARS_rkJRhW7MNs'),
 (99.51742553710938, 'PARS_xIhmbG5mYy'),
 (99.53881072998047, 'PARS_cM3cM2rTfb'),
 (99.54238891601562, 'PARS_FA2PTEudUH'),
 (99.55689239501953, 'PARS_NNounHRQS7'),
 (99.573486328125, 'PARS_i2oQELmrmW'),
 (99.59122467041016, 'PARS_5dVJPX6Ckj'),
 (99.59381103515625, 'PARS_EtGknjmkLS'),
 (99.59762573242188, 'PARS_udolVipaH8'),
 (99.63641357421875, 'PARS_C2F0J62t2R'),
 (99.64708709716797, 'PARS_ukjDIAEou4'),
 (99.65301513671875, 'PARS_TlLjhiEwTI'),
 (99.68982696533203, 'PARS_dMWIjKyGGY'),
 (99.72430419921875, 'PARS_miZ4brpYJJ'),
 (99.73123931884766, 'PARS_rEmQFfmcsc'),
 (99.74237060546875, 'PARS_O1MBVAQKXP'),
 (99.74324798583984

In [61]:
mrr = 0
for colans in list(colbert_answer.keys()):
    try:
        indices = [id_ for value, id_ in sorted(colbert_answer[colans], key=lambda x:x[0], reverse=True)].index(test_labels[colans])
        mrr += 1 / (indices + 1)
    except:
        pass

print(mrr / len(colbert_answer))

0.9524409258975747
